# YOLOv5 on SageMaker--推理

## 运行环境
Kernel 选择pytorch_latest_p36。  
本文在boto3 1.17.12和sagemaker 2.26.0下测试通过。

In [2]:
import boto3,sagemaker
print(boto3.__version__)
print(sagemaker.__version__)

1.17.12
2.26.0


## 本地推理(可选)
新启动一个shell窗口  
sudo mkdir /opt/ml  
sudo chmod 777 /opt/ml  
必须cd到`2-inference/source`目录，然后运行`python predictor.py`，正常启动会输出以下内容：
```
-------------init_output_dir  /opt/ml/output_dir
 * Serving Flask app "predictor" (lazy loading)
 * Environment: production
   WARNING: This is a development server. Do not use it in a production deployment.
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
```

In [ ]:
#修改请求图片
!curl -H "Content-Type: application/json" -X POST --data '{"bucket":"junzhong","image_uri":"yolov5/training/images/val/000799.jpeg"}' http://127.0.0.1:5000/invocations

## Amazon 深度学习容器

* [容器镜像清单](https://github.com/aws/deep-learning-containers/blob/master/available_images.md)
* 本文基于pytorch inference: `727897471807.dkr.ecr.cn-northwest-1.amazonaws.com.cn/pytorch-inference:1.6.0-gpu-py36-cu101-ubuntu16.04`

## Build自定义推理镜像

In [3]:
import boto3
region = boto3.session.Session().region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'yolov5-inference'
tag = ':latest'
uri_suffix = 'amazonaws.com'
if region in ['cn-north-1', 'cn-northwest-1']:
    uri_suffix = 'amazonaws.com.cn'
image_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)
print(image_uri)
ecr = '{}.dkr.ecr.{}.{}'.format(account_id, region, uri_suffix)

315505707008.dkr.ecr.cn-northwest-1.amazonaws.com.cn/yolov5-inference:latest


In [4]:
#国内pytorch inference基础镜像地址，不要修改
base_img='727897471807.dkr.ecr.cn-northwest-1.amazonaws.com.cn/pytorch-inference:1.6.0-gpu-py36-cu101-ubuntu16.04'
#登录基础镜像ECR，不要修改
!aws ecr get-login-password --region cn-northwest-1 | docker login --username AWS --password-stdin 727897471807.dkr.ecr.cn-northwest-1.amazonaws.com.cn

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [5]:
!aws ecr create-repository --repository-name $ecr_repository


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'yolov5-inference' already exists in the registry with id '315505707008'


In [6]:
%%time
!docker build -t $ecr_repository -f Dockerfile --build-arg BASE_IMG=$base_img .

Sending build context to Docker daemon   15.5MB
Step 1/12 : ARG BASE_IMG=${BASE_IMG}
Step 2/12 : FROM ${BASE_IMG}
 ---> a7cf51a6c06b
Step 3/12 : RUN apt-get update
 ---> Using cache
 ---> 6e0e40ab112d
Step 4/12 : RUN apt-get install -y --no-install-recommends nginx net-tools    && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> d1ae35fa3463
Step 5/12 : RUN pip install flask gevent gunicorn boto3 -i https://opentuna.cn/pypi/web/simple/ &&         rm -rf /root/.cache
 ---> Using cache
 ---> 80035286a458
Step 6/12 : WORKDIR /opt/ml/code
 ---> Using cache
 ---> 0facd849c209
Step 7/12 : COPY source ./
 ---> 88eee127fea7
Step 8/12 : RUN pip install -r requirements.txt -i https://opentuna.cn/pypi/web/simple/
 ---> Running in 03f46b87dc4e
Looking in indexes: https://opentuna.cn/pypi/web/simple/
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp36-cp36m-linux_x86_64.whl size=288126 sha256=3e5713ed0b9ab264cc5ac696559a1c0f019821da5049591f6cc81d378911a84c
  Stored in directory: /ro

In [ ]:
!docker tag $ecr_repository $image_uri
!$(aws ecr get-login --no-include-email)
!docker push $image_uri

## 部署模型到SageMaker

In [ ]:
from sagemaker.model import Model
import boto3

iam = boto3.client('iam')
roles = iam.list_roles(PathPrefix='/service-role')
role=""
for current_role in roles["Roles"]:
    if current_role["RoleName"].startswith("AmazonSageMaker-ExecutionRole-"):
        role=current_role["Arn"]
        break
#如果role为空表示有问题
print(role)

endpoint_name = "yolov5"
my_model = Model(
            role=role,
            image_uri=image_uri)

#该步骤大概需要15分钟
xgb_predictor = my_model.deploy(initial_instance_count=1,
                                endpoint_name=endpoint_name,
                                instance_type='ml.t2.medium'
                                )

## 推理

### 准备测试图片

In [ ]:
data = {"bucket":"junzhong","image_uri":"yolov5/training/images/val/000799.jpeg"}

返回的结果是对象的中心点位置(x,y)，以及宽高(w,h)

### 方式一、使用boto3

In [ ]:
import boto3
import json
endpoint_name = "yolov5"
client = boto3.client('runtime.sagemaker')

response = client.invoke_endpoint(EndpointName=endpoint_name,
                                  Body=json.dumps(data),
                                  ContentType="application/json")
#print(response)
response_body = response['Body'] 
body= response_body.read()
result = json.loads(body.decode('utf-8'))
print(result)

### 方式二、使用SageMaker SDK

In [ ]:
import sagemaker
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = sagemaker.predictor.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker.Session(),
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer())
result = predictor.predict(data)
print(result)

## 使用结果

### 定义函数

In [ ]:
!pip install tqdm -i https://pypi.tuna.tsinghua.edu.cn/simple/

In [ ]:
import sys
sys.path.append("source")
from IPython.display import Image
from utils.datasets import LoadImages
from utils.general import (xywh2xyxy, plot_one_box)
import cv2
from numpy import random
import torch
def xywh2xyxy(xywh,width,height):
    xyxy = []
    xyxy.append((xywh[0]-xywh[2]/2)*width)
    xyxy.append((xywh[1]-xywh[3]/2)*height)
    xyxy.append((xywh[0]+xywh[2]/2)*width)
    xyxy.append((xywh[1]+xywh[3]/2)*height)
    return xyxy

def draw(result,source_img,output_img):
    dataset = LoadImages(source_img)
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(50)]
    for path, img, im0s, vid_cap in dataset:
        shape = torch.tensor(im0s.shape)
        for item in result:
            xyxy_list = xywh2xyxy(item["xywh"],int(shape[1]),int(shape[0]))
            xyxy = []
            for xyxy_item in xyxy_list:
                xyxy.append(torch.Tensor([xyxy_item]))
            label = '%s %.2f' % (item["class_name"], item["confidence"])
            plot_one_box(xyxy, im0s, label=label, color=colors[item["class"]], line_thickness=3)
    cv2.imwrite(output_img, im0s)

### 下载原图并显示

In [ ]:
!mkdir result

In [ ]:
s3_file="s3://"+data["bucket"]+"/"+data["image_uri"]
!aws s3 cp $s3_file ./result
source_file_name=data["image_uri"].split("/")[-1:][0]
source_img="result/"+source_file_name
Image(filename=source_img, width=600)

### 把推理结果显示在图片上

In [ ]:
output_img="result/output_"+source_file_name
draw(result,source_img,output_img)
Image(filename=output_img, width=600)

## 删除Endpoint

In [ ]:
import boto3
sage = boto3.Session().client(service_name='sagemaker') 
sage.delete_endpoint(EndpointName=endpoint_name)
sage.delete_endpoint_config(EndpointConfigName=endpoint_name)